<a href="https://colab.research.google.com/github/Cicciokr/latin-ai-model/blob/main/02B.Fine-Tuning-Model-Text-Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
#!pip install evaluate
#!pip install rouge_score
!pip install wandb
!pip install transformers==4.48.3
#!pip install datasets==2.15.0
#!pip install evaluate==0.4.1
!pip install accelerate==0.27.2
#!pip install tqdm==4.66.1
!pip install peft==0.10.0
!pip install "numpy<2.0"


import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, AutoTokenizer, EarlyStoppingCallback, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
#from numba import cuda
import math
import torch
import wandb
import os

os.environ["WANDB_PROJECT"] = "Training-BARTLA"
wandb.login(key="e115f18967efc277b293c042cb216e776d82a741")
wandb.init()

MODEL_NAME = "Cicciokr/BART-CC100-la"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trucantion=True, model_max_length=1024)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


def preprocess_function(examples):
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=1024)
    tokens["labels"] = tokens["input_ids"].copy()  # Aggiunge i labels uguali agli input_ids
    return tokens

#"/kaggle/input/the-latin-library/thelatinlibrary_cleaned.txt"
dataset = load_dataset("text", data_files="/content/drive/MyDrive/Colab Notebooks/dataset/thelatinlibrary_cleaned.txt")
dataset_split = dataset['train'].train_test_split(train_size=0.8, test_size=0.2, shuffle=True)
#dataset_split_train = dataset['train'].train_test_split(test_size=0.0001, shuffle=True)
# Applicare la tokenizzazione
tokenized_datasets_test = dataset_split['test'].map(preprocess_function, batched=True, num_proc=4)
tokenized_datasets_train = dataset_split['train'].map(preprocess_function, batched=True, num_proc=4)

tokenized_datasets_test=tokenized_datasets_test.remove_columns(["text"])
tokenized_datasets_test.set_format("torch")

tokenized_datasets_train=tokenized_datasets_train.remove_columns(["text"])
tokenized_datasets_train.set_format("torch")

print(tokenized_datasets_train)
print(tokenized_datasets_test)

#il 20% dei dati viene usato come test e l'80% viene usato come train, per evitare overfitting
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/BART",
    run_name="bart-la-s",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    auto_find_batch_size=True,
    fp16=True,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Colab Notebooks/BART/logs",
    warmup_steps=10000,
    logging_steps=10000,
    save_total_limit=3,
    greater_is_better=True,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    report_to="wandb",
    metric_for_best_model="eval_loss",
    dataloader_num_workers=24
)
#xm.rendezvous("start_training")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    tokenizer=tokenizer
)

trainer.train(resume_from_checkpoint="/content/drive/MyDrive/Colab Notebooks/BART/checkpoint-55248")

trainer.save_model("/content/drive/MyDrive/Colab Notebooks/BART/bart-la-s")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/BART/bart-la-s-tokenizer")

wandb.finish()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cicciokr (cicciokr-universit-degli-studi-guglielmo-marconi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map (num_proc=4):   0%|          | 0/36831 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/147323 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 147323
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 36831
})


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-1-0f1fb4f3769b>:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker i

Epoch,Training Loss,Validation Loss


train/epoch,▁
train/global_step,▁
total_flos,2.6948461271187456e+17
train/epoch,3
train/global_step,55248
train_loss,0
train_runtime,15.2828
train_samples_per_second,28919.466
train_steps_per_second,3615.056


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
